# First, make a file name as train_chatbot.py. We import the necessary packages for our chatbot and initialize the variables we will use in our Python project.

# The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?','!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


# Preprocess data

# When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

# Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w,intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [5]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes),"classes",classes)
# words = all words,vocabulary
print(len(words),"unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


# Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [6]:
# Create our training data
training = []
# Create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag(for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag,output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5’.

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5',hist)
print("model created")

Train on 88 samples
Epoch 1/200
88/88 [==============================] - 9s 97ms/sample - loss: 1.0745 - accuracy: 0.7955
Epoch 2/200
88/88 [==============================] - 0s 3ms/sample - loss: 0.0064 - accuracy: 1.0000
Epoch 3/200
88/88 [==============================] - 0s 1ms/sample - loss: 0.0076 - accuracy: 1.0000
Epoch 4/200
88/88 [==============================] - 0s 982us/sample - loss: 0.0036 - accuracy: 1.0000
Epoch 5/200
88/88 [==============================] - 0s 1ms/sample - loss: 0.0016 - accuracy: 1.0000
Epoch 6/200
88/88 [==============================] - 0s 715us/sample - loss: 6.7132e-04 - accuracy: 1.0000
Epoch 7/200
88/88 [==============================] - 0s 827us/sample - loss: 9.1456e-04 - accuracy: 1.0000
Epoch 8/200
88/88 [==============================] - 0s 971us/sample - loss: 0.0014 - accuracy: 1.0000
Epoch 9/200
88/88 [==============================] - 0s 950us/sample - loss: 0.0011 - accuracy: 1.0000
Epoch 10/200
88/88 [==============================] 

88/88 [==============================] - 0s 749us/sample - loss: 2.1090e-04 - accuracy: 1.0000
Epoch 77/200
88/88 [==============================] - 0s 1ms/sample - loss: 2.4060e-04 - accuracy: 1.0000
Epoch 78/200
88/88 [==============================] - 0s 638us/sample - loss: 3.6566e-04 - accuracy: 1.0000
Epoch 79/200
88/88 [==============================] - 0s 553us/sample - loss: 2.5423e-05 - accuracy: 1.0000
Epoch 80/200
88/88 [==============================] - 0s 901us/sample - loss: 4.1688e-04 - accuracy: 1.0000
Epoch 81/200
88/88 [==============================] - 0s 1ms/sample - loss: 2.2222e-04 - accuracy: 1.0000
Epoch 82/200
88/88 [==============================] - 0s 917us/sample - loss: 2.1286e-05 - accuracy: 1.0000
Epoch 83/200
88/88 [==============================] - 0s 1ms/sample - loss: 1.4874e-04 - accuracy: 1.0000
Epoch 84/200
88/88 [==============================] - 0s 888us/sample - loss: 1.7644e-04 - accuracy: 1.0000
Epoch 85/200
88/88 [===========================

88/88 [==============================] - 0s 558us/sample - loss: 2.6371e-05 - accuracy: 1.0000
Epoch 152/200
88/88 [==============================] - 0s 729us/sample - loss: 9.9813e-06 - accuracy: 1.0000
Epoch 153/200
88/88 [==============================] - 0s 550us/sample - loss: 0.0023 - accuracy: 1.0000
Epoch 154/200
88/88 [==============================] - 0s 519us/sample - loss: 1.3793e-04 - accuracy: 1.0000
Epoch 155/200
88/88 [==============================] - 0s 630us/sample - loss: 2.1792e-05 - accuracy: 1.0000
Epoch 156/200
88/88 [==============================] - 0s 605us/sample - loss: 4.8414e-04 - accuracy: 1.0000
Epoch 157/200
88/88 [==============================] - 0s 660us/sample - loss: 6.0716e-05 - accuracy: 1.0000
Epoch 158/200
88/88 [==============================] - 0s 1ms/sample - loss: 4.8563e-06 - accuracy: 1.0000
Epoch 159/200
88/88 [==============================] - 0s 845us/sample - loss: 7.3891e-05 - accuracy: 1.0000
Epoch 160/200
88/88 [==================

# Now to predict the sentences and get a response from the user to let us create a new file ‘chatapp.py’.

# We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.

# Again we import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model:

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
    

# To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class.

In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details = True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence,model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list

In [10]:
# After predicting the class, we will get a random response from the list of intents.
def getResponse(ints,intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text,model)
    res = getResponse(ints,intents)
    return res

# Now we will code a graphical user interface. For this, we use the Tkinter library which already comes in python. We will take the input message from the user and then use the helper functions we have created to get the response from the bot and display it on the GUI. Here is the full source code for the GUI.

In [ ]:
# Creating GUI with tkinter
import tkinter
from tkinter import *

base = tkinter.Tk()
EntryBox = Text(base,bd=0, bg = "white", width = "30", height = "6", font = "TimesNewRoman")
msg = EntryBox.get("1.0",'end-1c').strip()

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
if msg != '':
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "You: " + msg + '\n\n')
    ChatLog.config(foreground = "#442265", font = ("Verdana", 13))
    
    res = chatbot_response(msg)
    ChatLog.insert(END, "Bot: " + res + '\n\n')
    
    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)
    

base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

# Create Chat window
ChatLog = Text(base, bd=0, bg = "white", height = "9", width = "51", font = "TimesNewRoman",)

ChatLog.config(state=DISABLED)

# Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command = ChatLog.yview, cursor = "heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Create Button to send message
SendButton = Button(base, font = ("Verdana",13,'bold'), text = "Send", width = "13", height=6, bd=0, bg = "#32de97", activebackground= "#3c9d9b", fg = '#ffffff', command = send)

# Create the box to enter message
EntryBox = Text(base,bd=0, bg = "white", width = "30", height = "6", font = "TimesNewRoman")

# EntryBox.bind("<Return>",send)

# Place all components on the screen
scrollbar.place(x=377,y=7,height=387)
ChatLog.place(x=7, y = 7, height = 387, width= 371)
EntryBox.place(x = 129, y = 402, height = 91, width = 266)
SendButton.place(x=7, y = 402, height = 91)

base.mainloop()